---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import numpy as np
import re
from datetime import datetime

# regex
months_ = 'January|February|March|April|May|June|July|August|September|October|November|December'
mons_ = 'Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Sep|Oct|Nov|Dec'

regex_ = r'(\d{0,2}[-/]?\d{0,2}[-/]\d{2,4})|((?:'+months_+'|'+mons_+')[-\s\.]?[\s]?\d{0,2}[stndrdh]{0,2}[-,]?[\s]?\d{2,4})|([12]\d{3})'

# date formats
date_formats = ['%m/%d/%Y', '%m-%d-%Y', '%m/%d/%y', '%m-%d-%y', '%m/%Y', '%b-%d-%Y', '%B-%d-%Y', '%B. %d, %Y',
                '%b %d %Y', '%b %d, %Y', '%B %d, %Y','%B %d %Y', '%b. %d, %Y', '%d %b %Y', '%d %B %Y', '%d %b. %Y',
                '%B %Y', '%b %Y', '%Y', '%B, %Y', '%b, %Y']

# function to extract date based on date formats
def extract_date(date_):
    for date_format in date_formats :
        try:
            parsed_date = datetime.strptime(date_, date_format)
            break 
        except ValueError:
            parsed_date = date_
            pass
    return parsed_date

In [3]:
def date_sorter():
    
    # pd.set_option('display.max_rows', 500)
    # https://regex101.com/r/niLalP/5
    # https://regex101.com/r/qauHKd/5
    result = pd.DataFrame(df.str.extract(regex_))
    dates_ = np.where(result[2].isnull(), np.where(result[1].isnull(), result[0], result[1]), result[2])
    
    # [80, 248, 271, 272, 417]
    # [print(value) for index, value in df[[39, 80, 99, 198, 204, 248, 271, 272, 417]].items()]
    dates_[80] = re.findall(regex_, df[80])[1][0]
    dates_[99] = re.findall(regex_, df[99])[0][0][1:] + re.findall(regex_, df[99])[1][0]
    dates_[248] = re.findall(regex_, df[248])[1][1]
    dates_[271] = re.findall(regex_, df[271])[2][1]
    dates_[272] = re.findall(regex_, df[272])[2][1]
    dates_[417] = re.findall(regex_, df[417])[0][0][1:]
    
    extracted_dates = pd.Series(list(map(extract_date, dates_)))
    sorted_index = pd.Series(extracted_dates.sort_values().index)
    
    return sorted_index